In [2]:
#!pip install pyspark

#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 43.0 MB/s eta 0:00:00
  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for pyspark ... done


In [1]:
import os
os.environ["JAVA_HOME"] = "/home/dariuslim269/jdk-11.0.2"


In [2]:
import gzip

gz_file_path = 'data/raw/green/2020/1/green_tripdata_2020_01.parquet.gz'
output_file_path = 'data/raw/green/2020/1/data.parquet'

# Define a function to decompress the .gz file
def decompress_gz(gz_file, output_file):
    with gzip.open(gz_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            f_out.write(f_in.read())

# Decompress the .gz file

for i in range(1,13,1):
    #decompress_gz(gz_file_path, output_file_path)
    gz_file_path = f'data/raw/green/2020/{i}/green_tripdata_2020_{i:02}.parquet.gz'
    output_file_path = f'data/raw/green/2020/{i}/data.parquet'
    decompress_gz(gz_file_path, output_file_path)

In [6]:
homework5_path = 'fhv_tripdata_2019-10'

In [43]:
!ls

data				jdk-11.0.2		 taxi+_zone_lookup.csv
download_script.sh		myApp.py		 taxi_zone_lookup.csv
fhv_tripdata_2019-10.parquet	myvenv			 zones
fhvhv				spark-3.3.2-bin-hadoop3  zoomcamp5.ipynb
fhvhv_tripdata_2021-01.parquet	spark-warehouse


In [9]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:/tmp/spark-events") \
    .getOrCreate()


df = spark.read \
    .option("header", "true") \
    .parquet('fhv_tripdata_2019-10.parquet')

df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|       264.0|       264.0|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|       264.0|       264.0|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|       264.0|       264.0|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [10]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropOff_datetime', TimestampNTZType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', IntegerType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [11]:
df = df.repartition(6)

In [12]:
df.write.parquet('fhvhv/2019/10', mode='overwrite')

In [13]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropOff_datetime: timestamp_ntz (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [14]:
#How many taxi trips were there on the 15th of October?
df.createOrReplaceTempView("homework5")

In [19]:
spark.sql(" WITH CTE as (SELECT pickup_datetime, DAY(pickup_datetime) AS day_timestamp FROM homework5) SELECT COUNT(*) FROM CTE WHERE day_timestamp = 15").show(truncate=False)

+--------+
|count(1)|
+--------+
|62629   |
+--------+



In [42]:
spark.sql(" WITH CTE AS (SELECT ABS(dropOff_datetime-pickup_datetime) AS duration FROM homework5), CTE2 as ( SELECT (extract(day FROM duration) * 24 +  \
    extract(hour FROM duration) + \
    extract(minute FROM duration) / 60 + \
    extract(second FROM duration) / 3600) AS duration_in_hours FROM CTE) \
    SELECT duration_in_hours FROM CTE2 order by duration_in_hours desc ").show(truncate=False)

+------------------+
|duration_in_hours |
+------------------+
|631152.5          |
|631152.5          |
|87672.44083333334 |
|70128.02805555555 |
|8794.0            |
|8784.166666666666 |
|1464.5344444444434|
|1056.8266666666666|
|1056.2705555555567|
|793.5530555555599 |
|793.3858333333334 |
|793.2975000000033 |
|792.9980555555534 |
|792.9883333333333 |
|792.86027777778   |
|792.8108333333299 |
|792.7850000000033 |
|792.7694444444467 |
|792.75388888889   |
|792.7463888888933 |
+------------------+
only showing top 20 rows



In [44]:
df2 = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

df2.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [55]:
df2.schema

StructType([StructField('LocationID', DoubleType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [53]:
from pyspark.sql.functions import col
df2 = df2.withColumn("LocationID", col("LocationID").cast("double"))

In [54]:
df2.createOrReplaceTempView("lookup")

In [51]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropOff_datetime', TimestampNTZType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', IntegerType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [47]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00310|2019-10-17 16:55:36|2019-10-17 16:59:14|       264.0|       208.0|   NULL|                B03017|
|              B01626|2019-10-05 02:25:14|2019-10-05 02:43:43|       264.0|       197.0|   NULL|                B01626|
|              B00789|2019-10-03 22:43:16|2019-10-03 22:52:21|       264.0|       264.0|   NULL|                B00789|
|              B00900|2019-10-09 10:59:32|2019-10-09 11:15:28|       264.0|       258.0|   NULL|                B00900|
|              B02157|2019-10-18 18:19:11|2019-10-18 18:21:10|       264.0|       265.0|   NULL|                B02157|
|              B03055|2019-10-18 13:55:5

In [59]:
spark.sql(" WITH CTE AS (SELECT * FROM homework5 a \
                        LEFT JOIN lookup b \
                        ON a.PUlocationID = b.LocationID) \
            SELECT COUNT(dispatching_base_num) as num,Zone \
            FROM CTE \
            GROUP BY Zone \
            ORDER BY num ASC").show()

+---+--------------------+
|num|                Zone|
+---+--------------------+
|  1|         Jamaica Bay|
|  2|Governor's Island...|
|  5| Green-Wood Cemetery|
|  8|       Broad Channel|
| 14|     Highbridge Park|
| 15|        Battery Park|
| 23|Saint Michaels Ce...|
| 25|Breezy Point/Fort...|
| 26|Marine Park/Floyd...|
| 29|        Astoria Park|
| 39|    Inwood Hill Park|
| 47|       Willets Point|
| 53|Forest Park/Highl...|
| 57|  Brooklyn Navy Yard|
| 62|        Crotona Park|
| 77|        Country Club|
| 89|     Freshkills Park|
| 98|       Prospect Park|
|105|     Columbia Street|
|110|  South Williamsburg|
+---+--------------------+
only showing top 20 rows

